In [1]:
%%time
import joblib, time, os, copy, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.nn import *
from torch.nn.functional import *

import torch
import torchvision

#enable it when running in google cloud, to upload/download file from/to VM to/from google cloud bucket
from google.cloud import storage

bucket_root_path = "weicheng30417"
project_data_folder = "data/breakfast-img-data/"

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)
torch.manual_seed(0)

cuda:0
CPU times: user 424 ms, sys: 44 ms, total: 468 ms
Wall time: 469 ms


In [2]:
def download_all_data_from_bucket():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    file_id = project_data_folder
    blobs=bucket.list_blobs(prefix=file_id)
    for blob in blobs:        
        file_names = blob.name.split("/")
        if len(file_names) == 3:
            if file_names[2] != "":
                file_name = file_names[2]            
                blob.download_to_filename(file_name)
                print("Download from {0} to local {1}".format(blob.name, file_name))
                

def download_file_from_bucket(file):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    butcketFile = project_data_folder + file
    blob = bucket.blob(butcketFile)
    blob.download_to_filename(file)
    print("Download from {0} to local {1}".format(butcketFile, file))
    
                
def upload_files(files):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    for file in files:
        butcketFile = project_data_folder + file
        blob = bucket.blob(butcketFile)
        blob.upload_from_filename(file)
        print("Upload from local {0} to {1}".format(file, butcketFile))

def upload_all_files():    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    
    directory = os.getcwd()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv") or filename.endswith(".ipynb") or filename.endswith(".model"):             
            butcketFile = project_data_folder + file
            blob = bucket.blob(butcketFile)
            blob.upload_from_filename(filename)
            print("Upload from local {0} to {1}".format(filename, butcketFile))        
        
        
def flatten(ls):
     for item in ls:
            for child in list(item):
                yield child

def get_segment_positions(x):
    ps = x.segment.split()
    ls = []
    for i in range(len(ps)-1):
        if i == 0:
            ls.append([int(ps[i]), int(ps[i+1])])
        else:
            ls.append([int(ps[i])+1, int(ps[i+1])])        
    return ls

def get_segment_features(x):
    ls = []
    for rg in x.positions:
        ls.append(x.feature[rg[0]:rg[1]])            
    return ls

def splitDataFrameList(df,target_column):
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df
                
def get_train_data():
    training_segment = pd.read_csv('training_segment.txt', header=None, names = ['segment'])
    training_segment['feature'] = joblib.load('train_feature.joblib')
    training_segment['positions'] = training_segment.apply(lambda x: get_segment_positions(x), axis=1)
    training_segment['feature'] = training_segment.apply(lambda x: get_segment_features(x), axis=1)
    training_segment = splitDataFrameList(training_segment, 'feature')
    training_segment['label'] = list(flatten(joblib.load('train_label.joblib')))
    training_segment = training_segment.drop(['segment','positions'], axis = 1)
    return training_segment

def get_test_data():
    test_segment = pd.read_csv('test_segment.txt', header=None, names = ['segment'])
    test_segment['feature'] = joblib.load('test_feature.joblib')
    test_segment['positions'] = test_segment.apply(lambda x: get_segment_positions(x), axis=1)
    test_segment['feature'] = test_segment.apply(lambda x: get_segment_features(x), axis=1)
    test_segment = splitDataFrameList(test_segment, 'feature')
    test_segment['ID'] = test_segment.index
    test_segment = test_segment.drop(['segment','positions'], axis = 1)
    return test_segment

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        ms = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:                
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)           

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)            
            print('{} Loss: {:.4f}, {} Acc: {:.4f}'.format(phase, epoch_loss, phase, epoch_acc))
        
        time_taken = str(datetime.timedelta(seconds=time.time() - ms))
        print('time taken: {}'.format(time_taken))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [3]:
%%time
train_df = get_train_data()
print(train_df.feature[0].shape)
print(train_df.info())

torch.Size([260, 400])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7075 entries, 0 to 7074
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feature  7075 non-null   object
 1   label    7075 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 110.7+ KB
None
CPU times: user 2.82 s, sys: 11.2 s, total: 14 s
Wall time: 36.3 s


In [16]:
%%time
train_sample_ratio = 1.0
train_data = train_df.sample(frac=train_sample_ratio)
train_data['feature_sum'] = train_data.apply(lambda x : torch.sum(x['feature'], dim=0).view(1,400), axis = 1)
train_data['feature_mean'] = train_data.apply(lambda x : torch.mean(x['feature'], dim=0).view(1,400), axis = 1)
train_data['label'] = train_data.apply(lambda x : x["label"] - 1, axis = 1)
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7075 entries, 633 to 6240
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   feature       7075 non-null   object
 1   label         7075 non-null   int64 
 2   feature_sum   7075 non-null   object
 3   feature_mean  7075 non-null   object
dtypes: int64(1), object(3)
memory usage: 276.4+ KB
None
CPU times: user 17.7 s, sys: 764 ms, total: 18.5 s
Wall time: 2.33 s


In [17]:
%%time
num_classes = 47
batch_size = 64
num_epochs = 10
feature_extract = True

train_feature = torch.stack(train_data['feature_mean'].tolist())
train_label = torch.tensor(train_data['label'].values.astype(np.long))

train_dataset = torch.utils.data.TensorDataset(train_feature, train_label)

train_size = int(0.8 * len(train_data))
val_size = int(0.1 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

dataloaders_dict = {'train':train_loader, 'val':test_loader}
print("len(train_loader.dataset) = ", len(train_loader.dataset))
print("len(val_loader.dataset) = ", len(val_loader.dataset))
print("len(test_loader.dataset) = ", len(test_loader.dataset))

len(train_loader.dataset) =  5660
len(val_loader.dataset) =  707
len(test_loader.dataset) =  708
CPU times: user 40 ms, sys: 12 ms, total: 52 ms
Wall time: 49.7 ms


In [21]:
%%time
class MyNet(Module):
    def __init__(self, batch_size, num_classes):
        super(MyNet, self).__init__()
        self.conv1 = Conv1d(in_channels=1, out_channels=batch_size, kernel_size=3, padding=1)
        self.conv1_bn = BatchNorm1d(num_features=batch_size)
        self.conv2 = Conv1d(in_channels=batch_size, out_channels=batch_size*2, kernel_size=3, padding=1)
        self.conv2_bn = BatchNorm1d(num_features=batch_size*2)
        self.conv3 = Conv1d(in_channels=batch_size*2, out_channels=batch_size*4, kernel_size=3, padding=1)
        self.conv3_bn = BatchNorm1d(num_features=batch_size*4)
        self.conv4 = Conv1d(in_channels=batch_size*4, out_channels=batch_size*8, kernel_size=3, padding=1)
        self.conv4_bn = BatchNorm1d(num_features=batch_size*8)
        self.pool = MaxPool1d(2, 2)
        self.drop1 = Dropout(0.5)
        self.linear1 = Linear(in_features=12800, out_features=batch_size*4)
        self.linear2 = Linear(in_features=batch_size*4, out_features=num_classes)
    
    def forward(self, x):
        x = self.pool(relu(self.conv1_bn(self.conv1(x))))
        #print("conv1", x.shape)
        x = self.pool(relu(self.conv2_bn(self.conv2(x))))
        #print("conv2",x.shape)
        x = self.pool(relu(self.conv3_bn(self.conv3(x))))
        #print("conv3",x.shape)
        x = self.pool(relu(self.conv4_bn(self.conv4(x))))
        #print("conv4",x.shape)
        x = self.drop1(x)
        
        #print("x = x.view(64, -1)", x.view(64, -1).shape)
        
        x = x.view(-1, 12800) 
        #print("x.view",x.shape)
        x = relu(self.linear1(x))
        #print("linear1",x.shape)
        x = self.drop1(x) 
        #x = softmax(self.linear2(x))
        x = self.linear2(x)
        return x


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)

model_ft = MyNet(batch_size, num_classes).double()
print(model_ft)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_ft = DataParallel(model_ft)

model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=False)

cuda:0
MyNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv1_bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2_bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3_bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4_bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=12800, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=47, bias=True)
)
Params to learn:
	 conv1.weight
	 conv1.bias
	 conv1_bn.we

In [23]:
%%time
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        inputs, labels = data[0], data[1]        
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test data: {0}'.format(100 * correct / total))

Accuracy of test data: 52.33380480905233
CPU times: user 104 ms, sys: 44 ms, total: 148 ms
Wall time: 143 ms


In [24]:
%%time
model_name = "model_4.0.model" #renamed to model_1.2.model
torch.save(model_ft.state_dict(), model_name)
upload_files([model_name])

Upload from local model_4.0.model to data/breakfast-img-data/model_4.0.model
CPU times: user 84 ms, sys: 88 ms, total: 172 ms
Wall time: 893 ms


In [25]:
%%time
batch_size = 64
num_classes = 47

model_name = "model_4.0.model"
model = MyNet(batch_size, num_classes).double().to(device)
model.load_state_dict(torch.load(model_name, map_location=device)) 

CPU times: user 792 ms, sys: 60 ms, total: 852 ms
Wall time: 133 ms


<All keys matched successfully>

In [32]:
%%time
test_df = get_test_data()
test_df['feature_sum'] = test_df.apply(lambda x : torch.sum(x['feature'], dim=0).view(1,400), axis = 1)
test_df['feature_mean'] = test_df.apply(lambda x : torch.mean(x['feature'], dim=0).view(1,400), axis = 1)
test_feature = torch.stack(test_df['feature_mean'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]
        inputs = inputs.to(device) 
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        result = np.append(result, predicted.cpu())

test_df['label'] = np.transpose(result).astype('int') 

CPU times: user 4.81 s, sys: 796 ms, total: 5.6 s
Wall time: 1.56 s


In [33]:
%%time
test_df['Category'] = test_df.apply(lambda x : x['label'] + 1, axis = 1)
test_df = test_df[['ID','Category']]
test_df

CPU times: user 36 ms, sys: 8 ms, total: 44 ms
Wall time: 43.5 ms


,ID,Category
0,0,22
1,1,2
2,2,2
3,3,10
4,4,22
...,...,...
1279,1279,44
1280,1280,14
1281,1281,15
1282,1282,2


In [34]:
test_df.Category.unique()

array([22,  2, 10, 21, 13, 15,  6, 27, 12, 32, 14, 33, 34, 44, 20, 11, 39,
        5, 45, 25, 19, 37, 46, 30, 24, 36, 29, 28, 26,  4,  1, 40, 43, 42,
       17,  3, 23,  7])

In [35]:
len(test_df.Category.unique())

38

In [37]:
submission_name = "submission_4.0_max.csv"
test_df.to_csv(submission_name, index=False)

In [38]:
upload_files([submission_name])

Upload from local submission_4.0_max.csv to data/breakfast-img-data/submission_4.0_max.csv
